# For more detailed report , refer to the attached pdf document

In [85]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
%cd drive/MyDrive
%cd Fetch\ Rewards\ CSV 

[Errno 2] No such file or directory: 'drive/MyDrive'
/content/drive/MyDrive/Fetch Rewards CSV
[Errno 2] No such file or directory: 'Fetch Rewards CSV'
/content/drive/MyDrive/Fetch Rewards CSV


# Importing Necessary Libraries

In [87]:
import pandas as pd
import numpy as np
import math

# Reading USERS table

In [88]:
users = pd.read_csv('users.csv')
users.head()

,_id__$oid,active,createdDate__$date,lastLogin__$date,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI


In [89]:
users.columns=["id","active","createdDate","lastLogin","role","signUpSource","state"]

# ISSUE 1 : Finding Duplicate Records

In [90]:
users.id.value_counts().sort_values(ascending=False)[:20]

54943462e4b07e684157a532    20
5fc961c3b8cfca11a077dd33    20
5ff5d15aeb7c7d12096d91a2    18
5fa41775898c7a11a6bcef3e    18
59c124bae4b0299e55b0f330    18
600fb1ac73c60b12049027bb    16
5ff1e194b6a9d73a3a9f1052    11
600987d77d983a11f63cfa92     9
600056a3f7e5b011fce897b0     8
5a43c08fe4b014fd6b6a0612     8
600f008f4329897eac237bd8     7
5ff4ce33c3d63511e2a484b6     7
60189c74c8b50e11d8454eff     7
5fff55dabd4dff11dda8f5f1     7
5ffc8f9704929111f6e922bf     6
600741d06e6469120a787853     6
6000b75bbe5fc96dfee1d4d3     6
5ff873d1b3348b11c9337716     5
6008893b633aab121bb8e0a9     5
60147d2ac8b50e11d8453f53     5
Name: id, dtype: int64

## Shape of original table

In [91]:
users.shape

(495, 7)

## Dropping duplicates

In [92]:
users_no_duplicates = users.copy()
users_no_duplicates = users_no_duplicates.drop_duplicates()


## Shape of non-duplicate data

In [93]:
users_no_duplicates.shape

(212, 7)

# ISSUE 2 : Removing ROLE column in USER Table

In [94]:
users_no_duplicates = users_no_duplicates.drop("role",axis=1)

In [95]:
users_no_duplicates.head()

,id,active,createdDate,lastLogin,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,Email,WI
6,5ff1e1e8cfcf6c399c274ad9,True,1609687528354,1.609688e+12,Email,WI
7,5ff1e1b7cfcf6c399c274a5a,True,1609687479626,1.609687e+12,Email,WI
9,5ff1e1f1cfcf6c399c274b0b,True,1609687537564,1.609688e+12,Email,WI


# Reading BRANDS Table

In [96]:
brands = pd.read_csv('brands.csv')
brands.head()

,_id__$oid,barcode,brandCode,category,categoryCode,cpg__$id__$oid,cpg__$ref,name,topBrand
0,601ac115be37ce2ead437551,511111019862,NaN,Baking,BAKING,601ac114be37ce2ead437550,Cogs,test brand @1612366101024,False
1,601c5460be37ce2ead43755f,511111519928,STARBUCKS,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Cogs,Starbucks,False
2,601ac142be37ce2ead43755d,511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146176,False
3,601ac142be37ce2ead43755a,511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146051,False
4,601ac142be37ce2ead43755e,511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,Cogs,test brand @1612366146827,False


In [97]:
brands.columns = ["id","barcode","brandCode","category","categoryCode","cpg_id","cpg_ref","name","topBrand"]

# ISSUE 3: Too many NULL values

## Number of missing values per column

In [98]:
brands.isnull().sum()

id                0
barcode           0
brandCode       269
category        155
categoryCode    650
cpg_id            0
cpg_ref           0
name              0
topBrand        612
dtype: int64

# ISSUE 4 : Numerous test values

## Creating subset containing test Entries

In [99]:
brands_test = brands[brands['name'].str.contains('test brand')]

In [100]:
brands_test.head()

,id,barcode,brandCode,category,categoryCode,cpg_id,cpg_ref,name,topBrand
0,601ac115be37ce2ead437551,511111019862,NaN,Baking,BAKING,601ac114be37ce2ead437550,Cogs,test brand @1612366101024,False
2,601ac142be37ce2ead43755d,511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146176,False
3,601ac142be37ce2ead43755a,511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146051,False
4,601ac142be37ce2ead43755e,511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,Cogs,test brand @1612366146827,False
5,601ac142be37ce2ead43755b,511111719885,TEST BRANDCODE @1612366146091,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146091,False


## Shape of original Dataset

In [101]:
brands.shape

(1167, 9)

## Category Frequency of original Dataset

In [102]:
brands.category.value_counts()

Baking                         369
Beer Wine Spirits               90
Snacks                          75
Candy & Sweets                  71
Beverages                       63
Magazines                       44
Health & Wellness               44
Breakfast & Cereal              40
Grocery                         39
Dairy                           33
Condiments & Sauces             27
Frozen                          24
Personal Care                   20
Baby                            18
Canned Goods & Soups            12
Beauty                           9
Cleaning & Home Improvement      6
Deli                             6
Beauty & Personal Care           6
Household                        5
Bread & Bakery                   5
Dairy & Refrigerated             5
Outdoor                          1
Name: category, dtype: int64

## Shape of test entries Dataset

In [103]:
brands_test.shape

(428, 9)

## Category Frequency of test entries dataset

In [104]:
brands_test.category.value_counts()

Baking            357
Candy & Sweets     71
Name: category, dtype: int64

In [105]:
brands_new=pd.concat([brands,brands_test]).drop_duplicates(keep=False)

## Dataset with no test entries

In [106]:
brands_new.head()

,id,barcode,brandCode,category,categoryCode,cpg_id,cpg_ref,name,topBrand
1,601c5460be37ce2ead43755f,511111519928,STARBUCKS,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Cogs,Starbucks,False
7,5cdad0f5166eb33eb7ce0faa,511111104810,J.L. KRAFT,Condiments & Sauces,NaN,559c2234e4b06aca36af13c6,Cogs,J.L. Kraft,NaN
8,5ab15636e4b0be0a89bb0b07,511111504412,CAMPBELLS HOME STYLE,Canned Goods & Soups,NaN,5a734034e4b0d58f376be874,Cogs,Campbell's Home Style,False
9,5c408e8bcd244a1fdb47aee7,511111504788,TEST,Baking,NaN,59ba6f1ce4b092b29c167346,Cogs,test,NaN
11,57c08106e4b0718ff5fcb02c,511111102540,NaN,NaN,NaN,5332f5f2e4b03c9a25efd0aa,Cpgs,MorningStar,NaN


## Shape of dataset with no test entries

In [107]:
brands_new.shape

(739, 9)

## Category Frequency of Dataset with no test entries

In [108]:
brands_new.category.value_counts()

Beer Wine Spirits              90
Snacks                         75
Beverages                      63
Health & Wellness              44
Magazines                      44
Breakfast & Cereal             40
Grocery                        39
Dairy                          33
Condiments & Sauces            27
Frozen                         24
Personal Care                  20
Baby                           18
Baking                         12
Canned Goods & Soups           12
Beauty                          9
Cleaning & Home Improvement     6
Deli                            6
Beauty & Personal Care          6
Household                       5
Bread & Bakery                  5
Dairy & Refrigerated            5
Outdoor                         1
Name: category, dtype: int64

# Issue 5 :  Columns have mixed datatypes in Brands table 

## Columns having numeric brandCode values have most values similar to barcode

In [109]:
brands[brands.brandCode.str.isnumeric()==True][:10]

,id,barcode,brandCode,category,categoryCode,cpg_id,cpg_ref,name,topBrand
13,5d6413156d5f3b23d1bc790a,511111205012,511111205012,Magazines,NaN,5d5d4fd16d5f3b23d1bc7905,Cogs,Entertainment Weekly,NaN
27,5d66d71fa3a018093ab34728,511111105329,511111105329,Magazines,NaN,5d5d4fd16d5f3b23d1bc7905,Cogs,Elegant Homes Magazine,NaN
44,5d66d94d6d5f3b6188d4f04b,511111505365,511111505365,Magazines,NaN,5d5d4fd16d5f3b23d1bc7905,Cogs,Magnolia Journal Magazine,NaN
64,5da609991dda2c3e1416ae90,511111805854,511111805854,Health & Wellness,NaN,53e10d6368abd3c7065097cc,Cogs,ONE A DAY® WOMENS,False
134,5da60576a60b87376833e349,511111305569,511111305569,Health & Wellness,NaN,53e10d6368abd3c7065097cc,Cogs,AFRIN® NO DRIP PUMP MISTS,NaN
137,5da608131dda2c3e1416ae8a,511111505716,511111505716,Health & Wellness,NaN,53e10d6368abd3c7065097cc,Cogs,Claritin® ADULTS,NaN
143,5d658ff3a3a018514994f432,511111005216,511111005216,Magazines,NaN,53e10d6368abd3c7065097cc,Cogs,Cooking Light Magazine,NaN
149,5d642dbfa3a018514994f42e,511111005148,511111005148,Magazines,NaN,5d5d4fd16d5f3b23d1bc7905,Cogs,Shape,NaN
152,5c45f91b87ff3552f950f027,511111204923,0987654321,Grocery,NaN,5c45f8b087ff3552f950f026,Cogs,Brand1,True
164,5da6094ca60b87376833e357,511111605829,511111605829,Health & Wellness,NaN,53e10d6368abd3c7065097cc,Cogs,ONE A DAY® MENS,NaN


# Reading Receipts data

## ISSUE 6 : Receipts data has rewardsreceiptItemsList which has nested column violating 1NF or atomicity

In [110]:
receipts = pd.read_csv('receipts.csv')
receipts.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (11,15,17,20,26,27,30,31,33,34,35,37,38,40,41,45,46,47,48,49,50,51,55,62,67,69,70,73,78,80,81,84,89,91,92,95,100,102,103,106,111,113,114,117,122,124,125,128,133,135,136,139,144,146,147,149,150,151,152,154,155,158,159,160,162,163,166,167,168,170,171,174,175,176,178,179,182,183,184,186,187,190,191,192,194,195,198,199,200,202,203,206,207,208,210,211,214,215,217,219,220,224,231,236,239,241,242,243,245,248,252,254,255,257,259,260,261,263,264,266,267,268,269,271,272,273,274,275,276,277,279,280,281,283,284,285,286,287,288,290,292,293,295,297,298,302,305,306,310,313,314,318,321,325,329,330,334,337,339,340,344,347,349,350,354,358,359,363,364,366,367,369,370,372,376,379,383,387,391,395,396,400,403,404,408,412,413,414,418,421,422,424,425,429,432,433,437,441,442,446,449,453,457,458,462,465,467,468,469,470,474,477,478,479,480,484,488,489,493,496,497,501,505,506,507,511,514,515,516,520,524,525,526,530

,_id/$oid,bonusPointsEarned,bonusPointsEarnedReason,createDate/$date,dateScanned/$date,finishedDate/$date,modifyDate/$date,pointsAwardedDate/$date,pointsEarned,purchaseDate/$date,...,rewardsReceiptItemList/458/competitorRewardsGroup,rewardsReceiptItemList/458/description,rewardsReceiptItemList/458/discountedItemPrice,rewardsReceiptItemList/458/finalPrice,rewardsReceiptItemList/458/itemPrice,rewardsReceiptItemList/458/needsFetchReview,rewardsReceiptItemList/458/originalReceiptItemText,rewardsReceiptItemList/458/partnerItemId,rewardsReceiptItemList/458/quantityPurchased,rewardsReceiptItemList/458/rewardsProductPartnerId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5.0,1.609632e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Columns: 6971 entries, _id/$oid to rewardsReceiptItemList/458/rewardsProductPartnerId
dtypes: float64(3446), int64(3), object(3522)
memory usage: 59.5+ MB


In [112]:
cols1  = receipts.iloc[: , :11]
cols2  = receipts.iloc[: , 23:26]

In [113]:
receipt_fin = pd.concat([cols1,cols2],axis=1)

In [114]:
receipt_fin.head()

,_id/$oid,bonusPointsEarned,bonusPointsEarnedReason,createDate/$date,dateScanned/$date,finishedDate/$date,modifyDate/$date,pointsAwardedDate/$date,pointsEarned,purchaseDate/$date,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5.0,1.609632e+12,1.0,REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,4.0,FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,2.0,FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


# Analyzing null Values

In [115]:
receipt_fin.isna().sum()

_id/$oid                     0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate/$date             0
dateScanned/$date            0
finishedDate/$date         551
modifyDate/$date             0
pointsAwardedDate/$date    582
pointsEarned               510
purchaseDate/$date         448
purchasedItemCount         484
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

In [116]:
receipt_fin.rewardsReceiptStatus.value_counts()

FINISHED     518
SUBMITTED    434
REJECTED      71
PENDING       50
FLAGGED       46
Name: rewardsReceiptStatus, dtype: int64

# ISSUE 7 : Receipts with Status "Submitted" has most of the columns null. Consumes Storage space

In [117]:
receipts_sub=receipt_fin[receipt_fin.rewardsReceiptStatus=='SUBMITTED']

In [118]:
receipts_sub.shape

(434, 14)

In [119]:
receipts_sub.isnull().sum()

_id/$oid                     0
bonusPointsEarned          434
bonusPointsEarnedReason    434
createDate/$date             0
dateScanned/$date            0
finishedDate/$date         434
modifyDate/$date             0
pointsAwardedDate/$date    434
pointsEarned               434
purchaseDate/$date         434
purchasedItemCount         434
rewardsReceiptStatus         0
totalSpent                 434
userId                       0
dtype: int64